# Goal
- Why? give a container access to host's USB Android devices. So that we can use one of [AndroidSDK docker](https://github.com/thyrlian/AndroidSDK#android-device) or [appium-docker-android](https://github.com/appium/appium-docker-android) on a Mac (e.g. OSX Catalina 10.15.7). So that we can map each device specifically to a docker-appium container and NOT necessarily have to give privileged access of the host to entire docker daemon using `docker run -it --privileged`
- To be able to correlate a USB device plugged into a Mac host to one of /dev/ttys*
- `ls /dev/tty*` - This command on mac lists all devices 


In [ ]:
# ! pip3 install plistlib
! pip3 install nest-asyncio

# This below cell just waits -- NOT CLEAR , but we part of this code below

In [ ]:
import asyncio
import plistlib
from typing import Iterable

import nest_asyncio
nest_asyncio.apply()
__import__('IPython').embed()


def devices_from_bus(bus_entries: Iterable[dict]):
    devices = []
    for bus_entry in bus_entries:
        if 'IORegistryEntryChildren' in bus_entry:
            devices.extend(
                devices_from_bus(bus_entry['IORegistryEntryChildren']))
        else:
            devices.append(bus_entry)
    return devices


async def probe_ioreg():
    process = await asyncio.create_subprocess_exec(
        'ioreg', '-p', 'IOUSB', '-a', '-l',
        stdout=asyncio.subprocess.PIPE)
    ioreg_stdout: bytes = (await process.communicate())[0]
    return plistlib.loads(ioreg_stdout, fmt=plistlib.FMT_XML)


def main():
    loop = asyncio.get_event_loop()
    ioreg = loop.run_until_complete(probe_ioreg())
    devices = devices_from_bus(ioreg['IORegistryEntryChildren'])
    for device in devices:
        if not device['Built-In']:
            print(f"Got device: {device['IORegistryEntryName']}")


if __name__ == '__main__':
    main()

# Execute the below command
- `ioreg -p IOUSB -a -l > blah.xml`

In [ ]:
import plistlib
with open("blah.xml", 'rb') as fp:
    ioreg = plistlib.load(fp)
ioreg

In [7]:
import asyncio
import plistlib
from collections.abc import Iterable

def devices_from_bus(bus_entries: Iterable[dict]):
    devices = []
    for bus_entry in bus_entries:
        if 'IORegistryEntryChildren' in bus_entry:
            devices.extend(
                devices_from_bus(bus_entry['IORegistryEntryChildren']))
        else:
            devices.append(bus_entry)
    return devices

devices = devices_from_bus(ioreg['IORegistryEntryChildren'])
for device in devices:
    if not device['Built-In']:
        print(f"Got device: {device['IORegistryEntryName']}")

Got device: USB 10/100/1000 LAN
Got device: USB2.0 Hub             
Got device: USB 2.0 BILLBOARD             
Got device: Nexus 6P


# More work 
- to understand the xml structure and ultimately tied "Nexus 6p" to /dev/ttys<number>
- ioreg -p IOUSB -a -l > blah.xml